In [1]:
import numpy as np
from matplotlib import pyplot as plt
import timeit

%matplotlib inline
%load_ext autoreload
%autoreload 2

import h5py
from pathlib import Path
from FKMC.jobmanager import *

## Test a single run first

In [3]:
%%time
from FKMC.quantum_montecarlo import quantum_cython_mcmc

config = dict(  N_steps = int(1e2),
                N_system = 8,
                mu = 0,
                beta = 10,
                V=-1,
                alpha=1.5,
                U = 0.1,
                output_state = False,
                output_correlator=True,
                output_history=False,
                quantum = True,
                job_id = 1020242,
               )

results = quantum_cython_mcmc(**config)

CPU times: user 104 ms, sys: 0 ns, total: 104 ms
Wall time: 103 ms


## Setup a new job from scratch

In [4]:
config = dict(
              mcmc_routine='quantum_cython_mcmc',
              N_steps = 10**4,
              N_system = 256,
              mu = 0,
              beta = np.linspace(0.1,2,100),
              U = 0.1,
              V= -1,
              alpha= 1.5,
              loop_over = np.array(['beta',], dtype=h5py.special_dtype(vlen=str)),
              quantum = True,
              output_history = False,
              output_correlator = True,
              output_state = False,
            )

working_dir = Path('./data/spin_spin_correlation_functions_using_quantum_simulator')
setup_mcmc(config, working_dir, overwrite = True)

config = read_config_file(working_dir)
config

INFO:FKMC.jobmanager:Working in: data/spin_spin_correlation_functions_using_quantum_simulator
INFO:FKMC.jobmanager:Sample results:
INFO:FKMC.jobmanager:classical_energies_moments: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:numbers_moments: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:magnetisations_moments: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:quantum_energies_moments: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:classical_energies_moments_err: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:numbers_moments_err: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:magnetisations_moments_err: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:quantum_energies_moments_err: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:correlator: array(shape=(129,), dtype=float64)
INFO:FKMC.jobmanager:q_energy_histogram: array(shape=(100,), dtype=float64)
INFO:FKMC.jobmanager:q_energy_histogram_bins: array(shape=(100,), dtype=float64)
INFO:F

{'mcmc_routine': 'quantum_cython_mcmc',
 'N_steps': 10000,
 'N_system': 256,
 'mu': 0,
 'beta': array([0.1       , 0.11919192, 0.13838384, 0.15757576, 0.17676768,
        0.1959596 , 0.21515152, 0.23434343, 0.25353535, 0.27272727,
        0.29191919, 0.31111111, 0.33030303, 0.34949495, 0.36868687,
        0.38787879, 0.40707071, 0.42626263, 0.44545455, 0.46464646,
        0.48383838, 0.5030303 , 0.52222222, 0.54141414, 0.56060606,
        0.57979798, 0.5989899 , 0.61818182, 0.63737374, 0.65656566,
        0.67575758, 0.69494949, 0.71414141, 0.73333333, 0.75252525,
        0.77171717, 0.79090909, 0.81010101, 0.82929293, 0.84848485,
        0.86767677, 0.88686869, 0.90606061, 0.92525253, 0.94444444,
        0.96363636, 0.98282828, 1.0020202 , 1.02121212, 1.04040404,
        1.05959596, 1.07878788, 1.0979798 , 1.11717172, 1.13636364,
        1.15555556, 1.17474747, 1.19393939, 1.21313131, 1.23232323,
        1.25151515, 1.27070707, 1.28989899, 1.30909091, 1.32828283,
        1.34747475, 1

In [28]:
config = dict(
              mcmc_routine='quantum_cython_mcmc',
              N_steps = 10**2,
              N_system = 32,
              mu = 0,
              beta = 1/np.linspace(5,0.1,5),
              U = np.linspace(0.1,10,5),
              V= np.array([0,-1,-10]),
              alpha= 1.5,
              loop_over = np.array(['V','beta','U'], dtype=h5py.special_dtype(vlen=str)),
              quantum = True,
              output_history = False,
              output_correlator = True,
              output_state = False,
            )

working_dir = Path('data/test')
setup_mcmc(config, working_dir, overwrite = True)

config = read_config_file(working_dir)
config

INFO:jobmanager:Working in: data/test
INFO:jobmanager:Sample results:
INFO:jobmanager:classical_energies_moments: array(shape=(9,), dtype=float64)
INFO:jobmanager:numbers_moments: array(shape=(9,), dtype=float64)
INFO:jobmanager:magnetisations_moments: array(shape=(9,), dtype=float64)
INFO:jobmanager:quantum_energies_moments: array(shape=(9,), dtype=float64)
INFO:jobmanager:classical_energies_moments_err: array(shape=(9,), dtype=float64)
INFO:jobmanager:numbers_moments_err: array(shape=(9,), dtype=float64)
INFO:jobmanager:magnetisations_moments_err: array(shape=(9,), dtype=float64)
INFO:jobmanager:quantum_energies_moments_err: array(shape=(9,), dtype=float64)
INFO:jobmanager:correlator: array(shape=(17,), dtype=float64)
INFO:jobmanager:q_energy_histogram: array(shape=(100,), dtype=float64)
INFO:jobmanager:runtime: array(shape=(1,), dtype=int64)


#!/usr/bin/env bash
#PBS -N test
#PBS -lselect=1:ncpus=1:mem=4gb:avx=true
#PBS -lwalltime=24:00:00
#PBS -J 1-75

echo ------------------------------------------------------
echo -n 'Job is running on node '; cat $PBS_NODEFILE
echo ------------------------------------------------------
echo PBS: qsub is running on $PBS_O_HOST
echo PBS: originating queue is $PBS_O_QUEUE
echo PBS: executing queue is $PBS_QUEUE
echo PBS: working directory is $PBS_O_WORKDIR
echo PBS: execution mode is $PBS_ENVIRONMENT
echo PBS: job identifier is $PBS_JOBID
echo PBS: job name is $PBS_JOBNAME
echo PBS: node file is $PBS_NODEFILE
echo PBS: current home directory is $PBS_O_HOME
echo PBS: PATH = $PBS_O_PATH
echo ------------------------------------------------------

module load intel-suite anaconda3/personal
. /home/tch14/anaconda3/etc/profile.d/conda.sh
conda activate idp

cd $HOME/Falicok-Kimball-Monte-Carlo/data/test
let "JOB_ID = PBS_ARRAY_INDEX - 1"
run_mcmc --job-id $JOB_ID --temp-dir $TMPDIR --working-di

{'mcmc_routine': 'quantum_cython_mcmc',
 'N_steps': 100,
 'N_system': 32,
 'mu': 0,
 'beta': array([ 0.2       ,  0.26490066,  0.39215686,  0.75471698, 10.        ]),
 'U': array([ 0.1  ,  2.575,  5.05 ,  7.525, 10.   ]),
 'V': array([  0,  -1, -10]),
 'alpha': 1.5,
 'loop_over': array(['V', 'beta', 'U'], dtype=object),
 'quantum': True,
 'output_history': False,
 'output_correlator': True,
 'output_state': False}

In [2]:
config = dict(
              mcmc_routine='quantum_cython_mcmc',
              N_burn_in = 10**3,
              N_steps = 10**4,
              N_system = 128,
              mu = 0,
              beta = np.logspace(-4,3,10),
              U = np.logspace(-2,2,10),
              V = np.array([-0.01,-0.05,-0.1,-0.15,-0.2,-0.5]),
              alpha= 1.5,
              loop_over = np.array(['V','beta','U'], dtype=h5py.special_dtype(vlen=str)),
              quantum = True,
              output_history = False,
              output_correlator = True,
              output_state = False,
            )

working_dir = Path('data/phase_diagram_6')
setup_mcmc(config, working_dir, overwrite = True)

config = read_config_file(working_dir)
config

INFO:FKMC.jobmanager:Working in: data/phase_diagram_6
INFO:FKMC.jobmanager:Sample results:
INFO:FKMC.jobmanager:flag_6: array(shape=(2,), dtype=int64)
INFO:FKMC.jobmanager:classical_energies_moments: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:numbers_moments: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:magnetisations_moments: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:quantum_energies_moments: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:classical_energies_moments_err: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:numbers_moments_err: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:magnetisations_moments_err: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:quantum_energies_moments_err: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:correlator: array(shape=(65,), dtype=float64)
INFO:FKMC.jobmanager:q_energy_histogram: array(shape=(500,), dtype=float64)
INFO:FKMC.jobmanager:IPR_histogram: array(shape=(500,), dtype=float6

ValueError: '/workspace/tch14/static_cx1_home/Falicov-Kimball-Monte-Carlo/notebooks/data/phase_diagram_6' does not start with '/workspace/tch14/cx1_home'

In [5]:
config = dict(
              mcmc_routine='quantum_cython_mcmc',
              N_steps = int(10**2),
              N_system = 128,
              mu = 0,
              beta = 0.6,
              U = np.array([1,2]),
              V = -0.1,
              alpha= 1.5,
              loop_over = np.array(['U'], dtype=h5py.special_dtype(vlen=str)),
              quantum = True,
              output_history = True,
              output_correlator = True,
              output_state = True,
            )

working_dir = Path('../data/test')
setup_mcmc(config, working_dir, overwrite = True)

config = read_config_file(working_dir)
config

INFO:FKMC.jobmanager:Working in: ../data/test
INFO:FKMC.jobmanager:Sample results:
INFO:FKMC.jobmanager:classical_energies_moments: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:numbers_moments: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:magnetisations_moments: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:quantum_energies_moments: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:classical_energies_moments_err: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:numbers_moments_err: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:magnetisations_moments_err: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:quantum_energies_moments_err: array(shape=(9,), dtype=float64)
INFO:FKMC.jobmanager:correlator: array(shape=(65,), dtype=float64)
INFO:FKMC.jobmanager:q_energy_histogram: array(shape=(100,), dtype=float64)
INFO:FKMC.jobmanager:q_energy_histogram_bins: array(shape=(100,), dtype=float64)
INFO:FKMC.jobmanager:classical_energies: array(shape=(1

{'mcmc_routine': 'quantum_cython_mcmc',
 'N_steps': 100,
 'N_system': 128,
 'mu': 0,
 'beta': 0.6,
 'U': array([1, 2]),
 'V': -0.1,
 'alpha': 1.5,
 'loop_over': array(['U'], dtype=object),
 'quantum': True,
 'output_history': True,
 'output_correlator': True,
 'output_state': True}

# Test a config before running on the cluster

In [ ]:
%%time
#working_dir = Path('./data/phase_diagram_2')
working_dir = Path('./data/spin_spin_correlation_functions_using_quantum_simulator')
#config = read_config_file(working_dir)
for job_id in [50]:
    run_mcmc(job_id, working_dir = working_dir, overwrite = True)

In [ ]:
config = read_config_file(working_dir)
incomplete = job_completion(working_dir)

print(working_dir)
print(f'Remaining: {len(incomplete)} Total: {total_jobs(config)}\n')
!qstat
!qstat -q

In [11]:
%%time
#working_dir = Path('./data/spin_spin_correlation_functions')
#working_dir = Path('./data/luijten_paper_Q_finer/')
#working_dir = Path('./data/spin_spin_correlation_functions_using_quantum_simulator')
working_dir = Path('./data/phase_diagram_5')
gather_mcmc(working_dir, do_all = False)

INFO:jobmanager:Number of Jobs to copy in: 10
INFO:jobmanager:Job IDs: [ 729  954 1060 1463 1464 1840 1904 1967 5123 5267]...
INFO:jobmanager:missing : [729, 954, 1060, 1464, 1840, 1904, 1967, 5123, 5267]
INFO:jobmanager:Jobs attempted this time: 10
INFO:jobmanager:Overall completion: 5391 / 5400
INFO:jobmanager:File size: 0.02Gb
CPU times: user 80.2 ms, sys: 8.39 ms, total: 88.6 ms
Wall time: 193 ms
